In [1]:
# Test the example in R for the network guided burden test.

In [21]:
library(rio)
library(tidyverse)
library(lmtest)

Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [4]:
ped = import('test.ped.txt')
head(ped)

ID,COV1,COV2,PHENO
ID1,1,2,0
ID3,2,3,0
ID4,1,3,0
ID5,2,1,1
ID2,0,5,1


In [5]:
genescore = import('test.gene.txt')
head(genescore)

T1,T2,T3,X,ID1,ID2,ID3,ID4,ID5,ID6
0,0,0,Y,0,0,0,1,1,1
0,0,0,T,0,0,1,1,1,1
0,0,0,M,1,0,1,1,1,1
0,0,0,Z,0,0,0,0,0,0


In [23]:
Y = genescore %>% filter(X=='Y') %>% select(!!ped[['ID']])
T = genescore %>% filter(X=='T') %>% select(!!ped[['ID']])
ped$Y = genescore %>% filter(X=='Y') %>% select(!!ped[['ID']]) %>% t(.)
ped$T = genescore %>% filter(X=='T') %>% select(!!ped[['ID']]) %>% t(.)
ped$M = genescore %>% filter(X=='M') %>% select(!!ped[['ID']]) %>% t(.)
ped

ID,COV1,COV2,PHENO,Y,T,M
ID1,1,2,0,0,0,1
ID3,2,3,0,0,1,1
ID4,1,3,0,1,1,1
ID5,2,1,1,1,1,1
ID2,0,5,1,0,0,0


In [19]:
g1 = glm(PHENO ~ Y + T + COV1 + COV2, data=ped, family = 'binomial')
summary(g1)


Call:
glm(formula = PHENO ~ Y + T + COV1 + COV2, family = "binomial", 
    data = ped)

Deviance Residuals: 
[1]  0  0  0  0  0

Coefficients:
              Estimate Std. Error z value Pr(>|z|)
(Intercept)    -466.76 1249762.74       0        1
Y               245.66  614494.86       0        1
T              -343.92  888558.07       0        1
COV1            245.66  668026.30       0        1
COV2             98.26  262021.47       0        1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6.7301e+00  on 4  degrees of freedom
Residual deviance: 2.1434e-10  on 0  degrees of freedom
AIC: 10

Number of Fisher Scoring iterations: 23


In [20]:
g0 =  glm(PHENO ~ COV1 + COV2, data=ped, family = 'binomial')
summary(g0)


Call:
glm(formula = PHENO ~ COV1 + COV2, family = "binomial", data = ped)

Deviance Residuals: 
      1        2        3        4        5  
-1.1335  -0.7355  -1.0551   1.5251   1.0942  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)
(Intercept)   1.1515     5.3784   0.214    0.830
COV1         -0.8746     2.0773  -0.421    0.674
COV2         -0.1905     1.1407  -0.167    0.867

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6.7301  on 4  degrees of freedom
Residual deviance: 6.4623  on 2  degrees of freedom
AIC: 12.462

Number of Fisher Scoring iterations: 4


In [22]:
lrtest(g0, g1)

#Df,LogLik,Df,Chisq,Pr(>Chisq)
3,-3.231168e+00,NA,NA,NA
5,-1.071667e-10,2,6.462336,0.03951133


In [24]:
g2 = glm(PHENO ~ M + T + COV1 + COV2, data=ped, family = 'binomial')
summary(g2)


Call:
glm(formula = PHENO ~ M + T + COV1 + COV2, family = "binomial", 
    data = ped)

Deviance Residuals: 
[1]  0  0  0  0  0

Coefficients:
              Estimate Std. Error z value Pr(>|z|)
(Intercept)  1.474e+02  4.814e+05       0        1
M           -1.228e+02  3.072e+05       0        1
T            2.457e+01  2.071e+05       0        1
COV1        -8.698e-15  1.853e+05       0        1
COV2        -2.457e+01  9.264e+04       0        1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 6.7301e+00  on 4  degrees of freedom
Residual deviance: 2.1434e-10  on 0  degrees of freedom
AIC: 10

Number of Fisher Scoring iterations: 23


In [25]:
lrtest(g0, g2)

#Df,LogLik,Df,Chisq,Pr(>Chisq)
3,-3.231168e+00,NA,NA,NA
5,-1.071667e-10,2,6.462336,0.03951133
